In [3]:
import tensorflow as tf
import pandas as pd

In [22]:
train_data = pd.read_csv('/content/Google_Stock_Price_Train.xls')

In [5]:
train_data.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [7]:
train_data.isna().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [9]:
train_data.shape

(1258, 6)

In [31]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))

In [32]:
train_col = train_data.iloc[:, 1:2]
scaled_train_col = scaler.fit_transform(train_col)

In [33]:
scaled_train_col[0:50, 0]

array([0.08581368, 0.09701243, 0.09433366, 0.09156187, 0.07984225,
       0.0643277 , 0.0585423 , 0.06568569, 0.06109085, 0.06639259,
       0.0614257 , 0.07474514, 0.02797827, 0.02379269, 0.02409033,
       0.0159238 , 0.01078949, 0.00967334, 0.01642607, 0.02100231,
       0.02280676, 0.02273235, 0.02810849, 0.03212665, 0.0433812 ,
       0.04475779, 0.04790163, 0.0440695 , 0.04648783, 0.04745517,
       0.04873875, 0.03936305, 0.04137213, 0.04034898, 0.04784582,
       0.04325099, 0.04356723, 0.04286033, 0.04602277, 0.05398467,
       0.05738894, 0.05714711, 0.05569611, 0.04421832, 0.04514845,
       0.04605997, 0.04412531, 0.03675869, 0.04486941, 0.05065481])

In [34]:
import numpy as np

X_train = np.array([scaled_train_col[i-50:i, 0] for i in range(50, len(scaled_train_col))])
y_train = np.array([scaled_train_col[i, 0] for i in range(50, len(scaled_train_col))])

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

## Simple RNN

In [36]:
regressor = tf.keras.Sequential(
    [
        tf.keras.layers.SimpleRNN(units = 50, return_sequences = True, input_shape = (50, 1)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.SimpleRNN(units = 50, return_sequences = True, input_shape = (50, 1)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.SimpleRNN(units = 50, return_sequences = True, input_shape = (50, 1)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.SimpleRNN(units = 50, return_sequences = True, input_shape = (50, 1)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units = 1)
    ]
)

In [37]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mean_squared_error'])
regressor.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_4 (SimpleRNN)    (None, 50, 50)            2600      
                                                                 
 dropout_8 (Dropout)         (None, 50, 50)            0         
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 50, 50)            5050      
                                                                 
 dropout_9 (Dropout)         (None, 50, 50)            0         
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 50, 50)            5050      
                                                                 
 dropout_10 (Dropout)        (None, 50, 50)            0         
                                                                 
 simple_rnn_7 (SimpleRNN)    (None, 50, 50)           

In [38]:
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
38/38 [==============================] - 8s 166ms/step - loss: 0.5058 - mean_squared_error: 0.5058
Epoch 2/100
38/38 [==============================] - 6s 160ms/step - loss: 0.2769 - mean_squared_error: 0.2769
Epoch 3/100
38/38 [==============================] - 6s 162ms/step - loss: 0.2046 - mean_squared_error: 0.2046
Epoch 4/100
38/38 [==============================] - 6s 167ms/step - loss: 0.1578 - mean_squared_error: 0.1578
Epoch 5/100
38/38 [==============================] - 6s 161ms/step - loss: 0.1319 - mean_squared_error: 0.1319
Epoch 6/100
38/38 [==============================] - 6s 158ms/step - loss: 0.1158 - mean_squared_error: 0.1158
Epoch 7/100
38/38 [==============================] - 6s 157ms/step - loss: 0.1048 - mean_squared_error: 0.1048
Epoch 8/100
38/38 [==============================] - 6s 162ms/step - loss: 0.0972 - mean_squared_error: 0.0972
Epoch 9/100
38/38 [==============================] - 6s 160ms/step - loss: 0.0930 - mean_squared_error: 0.0930
E

## LSTM 

In [39]:
lstm_regressor = tf.keras.Sequential(
    [
        tf.keras.layers.LSTM(units = 50, return_sequences = True, input_shape = (50, 1)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(units = 50, return_sequences = True, input_shape = (50, 1)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(units = 50, return_sequences = True, input_shape = (50, 1)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(units = 50, return_sequences = True, input_shape = (50, 1)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units = 1)
    ]
)

In [40]:
lstm_regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mean_squared_error'])
lstm_regressor.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 50, 50)            10400     
                                                                 
 dropout_12 (Dropout)        (None, 50, 50)            0         
                                                                 
 lstm_5 (LSTM)               (None, 50, 50)            20200     
                                                                 
 dropout_13 (Dropout)        (None, 50, 50)            0         
                                                                 
 lstm_6 (LSTM)               (None, 50, 50)            20200     
                                                                 
 dropout_14 (Dropout)        (None, 50, 50)            0         
                                                                 
 lstm_7 (LSTM)               (None, 50, 50)           

In [41]:
lstm_regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
38/38 [==============================] - 6s 15ms/step - loss: 0.1238 - mean_squared_error: 0.1238
Epoch 2/100
38/38 [==============================] - 0s 13ms/step - loss: 0.0877 - mean_squared_error: 0.0877
Epoch 3/100
38/38 [==============================] - 0s 13ms/step - loss: 0.0846 - mean_squared_error: 0.0846
Epoch 4/100
38/38 [==============================] - 1s 13ms/step - loss: 0.0821 - mean_squared_error: 0.0821
Epoch 5/100
38/38 [==============================] - 0s 13ms/step - loss: 0.0818 - mean_squared_error: 0.0818
Epoch 6/100
38/38 [==============================] - 0s 13ms/step - loss: 0.0804 - mean_squared_error: 0.0804
Epoch 7/100
38/38 [==============================] - 0s 12ms/step - loss: 0.0796 - mean_squared_error: 0.0796
Epoch 8/100
38/38 [==============================] - 0s 13ms/step - loss: 0.0791 - mean_squared_error: 0.0791
Epoch 9/100
38/38 [==============================] - 0s 13ms/step - loss: 0.0785 - mean_squared_error: 0.0785
Epoch 10/1

Both the simple RNN and the LSTM model have similar mean_squared_error. But since the LSTM model selectively remembers previous state, its training speed is much higher than the simple RNN